In [ ]:
!pip install -e ../.
from flytekit.configuration import set_flyte_config_file
set_flyte_config_file("notebook.config")

In [1]:
import boto3
from botocore.client import Config

s3 = boto3.resource('s3',
                    endpoint_url='http://localhost:30084',
                    aws_access_key_id='minio',
                    aws_secret_access_key='miniostorage',
                    config=Config(signature_version='s3v4'),
                    region_name='us-east-1')

# Demo: quick remote iteration with an OpenCV Script

> filter_edges

In [2]:
from image import filter_edges

In [3]:
from IPython.display import Image, display
def display_images(paths):
    for p in paths:
        display(Image(p))

In [ ]:
inp = "image.jpg"
out = "edges.png"
filter_edges(inp, out)
display_images([inp, out])

In [ ]:
from flytekit.common.tasks.raw_container import SdkRawContainerTask
from flytekit.sdk.types import Types
from flytekit.sdk.workflow import workflow_class, Input, Output

edges = SdkRawContainerTask(
    input_data_dir="/inputs",
    output_data_dir="/outputs",
    inputs={"image": Types.Blob, "script": Types.Blob},
    outputs={"edges": Types.Blob},
    image="jjanzic/docker-python3-opencv",
    command=["python", "/inputs/script", "/inputs/image", "/outputs/edges"],
)


@workflow_class
class EdgeDetector(object):
    script = Input(Types.Blob)
    image = Input(Types.Blob)
    edge_task = edges(script=script, image=image)
    out =  Output(edge_task.outputs.edges, sdk_type=Types.Blob)

EdgeDetector_lp = EdgeDetector.create_launch_plan()    

In [ ]:
edges.register(name="EdgeDetectorFunc", project="flyteexamples", domain="development", version="5")
EdgeDetector.register(name="EdgeDetector", project="flyteexamples", domain="development", version="5")
EdgeDetector_lp.register(name="EdgeDetector", project="flyteexamples", domain="development", version="5")

In [ ]:
image = "https://www.naturephotographysimplified.com/wp-content/uploads/2019/06/How-to-get-sharp-images-Birds-in-flight-Bharatpur-Bird-Sanctuary-bird-Photography-by-Prathap-DK-bronze-winged-jacana-Greater-Spotted-Eagle-750x500.jpg"
s3.Bucket('my-s3-bucket').upload_file('image.py','code/image.py')
exc = EdgeDetector_lp.execute("flyteexamples", "development", inputs={"script": "s3://my-s3-bucket/code/image.py", "image": image})
print("http://localhost:30081/console/projects/flyteexamples/domains/development/executions/{}".format(exc.id.name))

In [ ]:
key="/gn/ff554920363ff4da1903-edge-task-0/edges"
s3.Bucket('my-s3-bucket').download_file(key,'edges.png')

In [ ]:
display_images(["edges.png"])

In [ ]:
import cv2
img = cv2.imread("image.jpg", 0)

In [ ]:
img